In [1]:
#pip install lightgbm
#!pip install xgboost
#!pip install --upgrade notebook
#pip install ipywidgets
#!pip install notebook==6.5.4

In [2]:
#importing the Libraries#
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [3]:
dataset1=pd.read_csv("churn_preprocess_data.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)
df2.replace({True: 1, False: 0}, inplace=True)

indep_X=df2.drop('Churn_Yes',axis=1)
dep_Y=df2['Churn_Yes']

In [4]:
df2.columns


Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit card', 'PaymentMethod_Electronic check',
       'PaymentMethod_Mailed check', 'Churn_Yes'],
      dtype='object')

In [5]:
indep_X

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card,PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,5,27.43,137.15,1,1,1,0,0,1,...,0,0,1,0,0,0,1,1,0,0
1,0,42,38.28,1607.76,0,0,0,1,1,0,...,1,0,1,0,0,1,0,0,1,0
2,0,61,106.44,6492.84,1,0,0,1,1,0,...,0,1,0,0,1,0,0,0,0,0
3,1,22,92.49,2034.78,1,0,1,1,0,0,...,1,0,0,1,0,0,0,0,1,0
4,1,21,19.63,412.23,1,1,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,61,42.52,2593.72,0,0,1,0,0,1,...,0,0,0,0,1,0,1,0,1,0
296,0,49,22.31,1093.19,0,0,0,1,1,0,...,0,1,0,1,0,0,1,0,0,1
297,0,71,59.57,4229.47,1,0,1,1,0,1,...,0,0,0,1,0,1,0,0,0,1
298,1,1,31.32,31.32,1,1,1,1,0,1,...,0,1,0,1,0,1,0,1,0,0


In [6]:
dep_Y

0      1
1      1
2      1
3      0
4      1
      ..
295    0
296    1
297    1
298    0
299    1
Name: Churn_Yes, Length: 300, dtype: uint8

In [7]:
#machine learning algorithms provider is sklearn #split train set and test set#
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test=train_test_split(indep_X, dep_Y,test_size=0.30,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
def selectkbest(indep_X, dep_Y, n):
    selectk_features = []
    selected_columns = []
    
    test = SelectKBest(score_func=chi2, k=n)  # chi2 function use panren
    fit1 = test.fit(indep_X, dep_Y)

    # Correct way to get selected feature names
    selected_features = indep_X.columns[test.get_support()].tolist()
    selected_columns.append(selected_features)  # Store selected features

    # Print selected features
    print("\nSelected Features:")
    for i, model_features in enumerate(selected_columns):
        print(f" Model {i+1}: {model_features}")  #

    selectk_features = test.transform(indep_X)
    return selectk_features, selected_columns
    

In [10]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def lightgbm_model(X_train, y_train, X_test):

    # Fitting lightgbm to the Training set
    from lightgbm import LGBMClassifier
    classifier = LGBMClassifier(random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm


def xgboost_model(X_train, y_train, X_test):

     # Fitting xgboost to the Training set
    from xgboost import XGBClassifier
    classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
    
def selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf,acclgbm,accxgb): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random',
                                                        'lightgbm_model','XGBoost'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
        dataframe['lightgbm_model'][idex]=acclgbm[number]
        dataframe['XGBoost'][idex]=accxgb[number]
    return dataframe

In [11]:
selectk_features,selected_columns=selectkbest(indep_X,dep_Y,6)       

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
acclgbm=[]
accxgb=[]


Selected Features:
 Model 1: ['tenure', 'MonthlyCharges', 'TotalCharges', 'OnlineSecurity_Yes', 'StreamingTV_No internet service', 'PaymentMethod_Mailed check']


In [12]:
for i in selectk_features:   
   X_train, X_test, y_train, y_test=split_scalar(selectk_features,dep_Y)   
    
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)

classifier, Accuracy, report, X_test, y_test, cm = lightgbm_model(X_train, y_train, X_test)
acclgbm.append(Accuracy)

classifier, Accuracy, report, X_test, y_test, cm = xgboost_model(X_train, y_train, X_test)
accxgb.append(Accuracy)
    
# Store accuracies in dictionary
model_accuracies = {
    "Logistic Regression": acclog[0],
    "SVM (Linear)": accsvml[0],
    "SVM (Non-Linear)": accsvmnl[0],
    "KNN": accknn[0],
    "Naive Bayes": accnav[0],
    "Decision Tree": accdes[0],
    "Random Forest": accrf[0],
    "LightGBM": acclgbm[0],
    "XGBoost": accxgb[0]
}

# Find best model
best_model = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model]

# Print best model with accuracy
print(f"Best Model: {best_model} with Accuracy: {best_accuracy:.2f}")


C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[LightGBM] [Info] Number of positive: 108, number of negative: 117
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 217
[LightGBM] [Info] Number of data points in the train set: 225, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480000 -> initscore=-0.080043
[LightGBM] [Info] Start training from score -0.080043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\xgboost\core.py:158: UserWarning: [15:38:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [13]:
#Model creation#
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define parameter grid
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],  # Regularization type
    'C': [0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga'],  # Optimizer
    'max_iter': [100, 200, 500]  # Number of iterations
}

# Create logistic regression model
log_reg = LogisticRegression()
    
grid=GridSearchCV(log_reg, param_grid, refit=True, verbose=3,n_jobs=-1,scoring='f1_weighted')

#fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.35585678 0.34104782 0.58803717 0.60184595        nan        nan
        nan        nan 0.35585678 0.35091872 0.58803717 0.60184595
        nan        nan        nan        nan 0.35585678 0.34104782
 0.58803717 0.60184595        nan        nan        nan        nan
 0.48881012 0.44643998 0.56683148 0.57519386        nan        nan
        nan        nan 0.48881012 0.44643998 0.56683148 0.57519386
        nan        nan        nan        nan 0.48881012 0.44643998
 0.56683148 0.57519386        nan        nan        nan        nan
 0.56134325 0.56057398 0.56498219 0.56498219        nan        nan
        nan        nan 0.56134325 0.56057398 0.56498219 0.56498219
        nan        nan        nan        nan 0.56134325 0.56057398
 0.56498219 0.56498219        nan        nan        nan        nan
 0.56556351 0.56556351 0.56971236 0

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 500],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning 

re=grid.cv_results_
y_pred = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)

C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anandha rishi\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 0.01, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}: 0.43383753501400557


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[44  0]
 [31  0]]


In [17]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.59      1.00      0.74        44
           1       0.00      0.00      0.00        31

    accuracy                           0.59        75
   macro avg       0.29      0.50      0.37        75
weighted avg       0.34      0.59      0.43        75



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.5

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003126,0.006251,0.006251,0.007655,0.01,100,l1,liblinear,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l1', ...",0.371014,0.371014,0.345752,0.345752,0.345752,0.355857,0.012376,-2147483648
1,0.003119,0.006239,0.009381,0.007660,0.01,100,l1,saga,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l1', ...",0.296970,0.371014,0.345752,0.345752,0.345752,0.341048,0.024113,-2147483648
2,0.000000,0.000000,0.009376,0.007656,0.01,100,l2,liblinear,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.608485,0.576937,0.577360,0.555556,0.621848,0.588037,0.023890,-2147483648
3,0.003125,0.006251,0.003125,0.006250,0.01,100,l2,saga,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.600532,0.594332,0.657148,0.565720,0.591498,0.601846,0.030095,-2147483648
4,0.003126,0.006251,0.000000,0.000000,0.01,100,elasticnet,liblinear,"{'C': 0.01, 'max_iter': 100, 'penalty': 'elast...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2147483648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.009376,0.007655,0.003126,0.006252,100,500,l2,saga,"{'C': 100, 'max_iter': 500, 'penalty': 'l2', '...",0.549258,0.576937,0.619966,0.551111,0.575256,0.574505,0.025526,-2147483648
116,0.000000,0.000000,0.000000,0.000000,100,500,elasticnet,liblinear,"{'C': 100, 'max_iter': 500, 'penalty': 'elasti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2147483648
117,0.003125,0.006249,0.000000,0.000000,100,500,elasticnet,saga,"{'C': 100, 'max_iter': 500, 'penalty': 'elasti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2147483648
118,0.000000,0.000000,0.000000,0.000000,100,500,None,liblinear,"{'C': 100, 'max_iter': 500, 'penalty': None, '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2147483648


In [21]:
columns = df2.columns  # DataFrame columns list
##['Partner_Yes', 'MultipleLines_No phone service', 'OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'Contract_Two year', 'PaymentMethod_Credit card']

for col in columns:
    var_name = col.replace(" ", "_")  # Replace spaces with underscores
    dtype = "int" if "Yes" in col or "No" in col else "float" if "Charges" in col else "input"
    print(f'{var_name} = {dtype}(input("{var_name}: "))')

SeniorCitizen = input(input("SeniorCitizen: "))
tenure = input(input("tenure: "))
MonthlyCharges = float(input("MonthlyCharges: "))
TotalCharges = float(input("TotalCharges: "))
gender_Male = input(input("gender_Male: "))
Partner_Yes = int(input("Partner_Yes: "))
Dependents_Yes = int(input("Dependents_Yes: "))
PhoneService_Yes = int(input("PhoneService_Yes: "))
MultipleLines_No_phone_service = int(input("MultipleLines_No_phone_service: "))
MultipleLines_Yes = int(input("MultipleLines_Yes: "))
InternetService_Fiber_optic = input(input("InternetService_Fiber_optic: "))
InternetService_No = int(input("InternetService_No: "))
OnlineSecurity_No_internet_service = int(input("OnlineSecurity_No_internet_service: "))
OnlineSecurity_Yes = int(input("OnlineSecurity_Yes: "))
OnlineBackup_No_internet_service = int(input("OnlineBackup_No_internet_service: "))
OnlineBackup_Yes = int(input("OnlineBackup_Yes: "))
DeviceProtection_No_internet_service = int(input("DeviceProtection_No_internet_service: ")

In [22]:
tenure = int(input("tenure:"))
MonthlyCharges = float(input("MonthlyCharges:"))
TotalCharges = float(input("TotalCharges:"))
OnlineSecurity_Yes = int(input("OnlineSecurity_Yes:"))
StreamingMovies_No_internet_service = int(input("StreamingMovies_No_internet_service:"))
PaymentMethod_Mailed_check = int(input("PaymentMethod_Mailed_check:"))

tenure: 23
MonthlyCharges: 34
TotalCharges: 45
OnlineSecurity_Yes: 56
StreamingMovies_No_internet_service: 67
PaymentMethod_Mailed_check: 78


In [23]:
Final_prediction =grid.predict([[tenure,MonthlyCharges,TotalCharges,OnlineSecurity_Yes,StreamingMovies_No_internet_service,PaymentMethod_Mailed_check]])
print("Final_prediction of Logistic Regression: {}".format(Final_prediction))

Final_prediction of Logistic Regression: [0]


In [24]:
import pickle
# Save the best model

with open("best_model.sav", "wb") as file:
    pickle.dump(grid.best_estimator_, file)

print("Model saved successfully as 'best_model.sav'")

Model saved successfully as 'best_model.sav'


In [25]:
import pickle
filename="best_model.sav.sav"

In [26]:
pickle.dump(grid,open(filename,'wb'))
load_model=pickle.load(open("best_model.sav",'rb'))

In [27]:
def get_categorical_prediction(prediction):
    return "Yes" if prediction == 1 else "No"
tenure = int(input("tenure:"))
MonthlyCharges = float(input("MonthlyCharges:"))
TotalCharges = float(input("TotalCharges:"))
OnlineSecurity_Yes = int(input("OnlineSecurity_Yes:"))
StreamingMovies_No_internet_service = int(input("StreamingMovies_No_internet_service:"))
PaymentMethod_Mailed_check = int(input("PaymentMethod_Mailed_check:"))

tenure: 23
MonthlyCharges: 34
TotalCharges: 45
OnlineSecurity_Yes: 56
StreamingMovies_No_internet_service: 67
PaymentMethod_Mailed_check: 78


In [28]:
Final_result = load_model.predict([[tenure,MonthlyCharges,TotalCharges,OnlineSecurity_Yes,StreamingMovies_No_internet_service,PaymentMethod_Mailed_check]])
Final_result

array([0], dtype=uint8)

In [29]:
future_prediction_categorical = get_categorical_prediction(Final_result)
future_prediction_categorical

'No'

In [30]:
if future_prediction_categorical == "Yes":
    print("Yes, this customer is going to stop using the service.")
else:
    print("No, this customer will continue using the service.")

No, this customer will continue using the service.
